In [1]:
# Package import

import cv2
import numpy as np
import smtplib
import playsound
import threading

In [2]:
# Sound alarm
def play_alarm_sound_function():
	while True:
		playsound.playsound('./alarm-sound.mp3',True)

In [3]:
# Send email
def send_mail_function():

    recipientEmail = "cuscjesus@gmail.com"
    recipientEmail = recipientEmail.lower()

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.ehlo()
        server.starttls()
        server.login('cuscjesus@gmail.com', 'Pa$$word123')
        server.sendmail('cuscjesus@gmail.com', recipientEmail, "Warning A Fire Accident has been reported on Your Company")
        print("sent to {}".format(recipientEmail))
        server.close()
    except Exception as e:
    	print(e)


In [47]:
#Variable declaration
Alarm_Status = False
Email_Status = False
Fire_Reported = 0

video = cv2.VideoCapture("./swing.mp4") # If you want to use webcam use Index like 0,1.

while True:
    (grabbed, frame) = video.read()
    if not grabbed:
        break

    frame = cv2.resize(frame, (960, 540))
    frame = cv2.flip(frame, 1)

    blur = cv2.GaussianBlur(frame, (7, 7), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    lower = [11, 33, 111]
    upper = [90, 255, 255]
    lower = np.array(lower, dtype="uint8")
    upper = np.array(upper, dtype="uint8")

    mask = cv2.inRange(hsv, lower, upper)

    output = cv2.bitwise_and(frame, hsv, mask=mask)

    no_red = cv2.countNonZero(mask)

    if int(no_red) > 20000:
        Fire_Reported = Fire_Reported + 1

    cv2.imshow("output", output)

    if Fire_Reported >= 1:
        
        if Alarm_Status == False:
            print("fire")
            #threading.Thread(target=play_alarm_sound_function).start()
            Alarm_Status = True

        if Email_Status == False:
            #threading.Thread(target=send_mail_function).start()
            Email_Status = True

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
video.release()